In [1]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

2023-11-29 13:20:27.204706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 13:20:27.204747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 13:20:27.206053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 13:20:27.212931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 13:20:28.121881: W tensorflow/compiler/tf2

In [2]:
! pip install matplotlib


In [3]:
! pip install tensorflow

In [4]:
! pip install nvidia-tensorrt

In [5]:
# Import necessary libraries
import matplotlib
matplotlib.use("Agg")

import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import VGG16
from keras.optimizers import Adam
from keras.layers import Flatten, Dense, Dropout, Input
from keras.models import Model
import numpy as np

from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam

In [6]:
# Directory paths

train_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/train'
valid_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/train'
test_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/test'

In [7]:
# CLASSES = 6       # There are 6 classes of defects in each directory: Crazing, Inclusion, Patches, Pitted, Rolled, Scratches
# HEIGHT = 224      # Height of the input image for the model
# WIDTH = 224       # Width of the input image for the model
# CHANNELS = 3      # Number of color channels in the image (RGB - Red, Green, Blue)
# BATCH_SIZE = 32   # Number of images processed in a single batch during training or inference
# Epochs = 5


HEIGHT = 224
WIDTH = 224
CHANNELS = 3
CLASSES = 6


In [8]:

baseModel = MobileNetV2(input_shape=(HEIGHT, WIDTH, CHANNELS),
                        include_top=False,
                        weights='imagenet')

In [9]:
# # Create a VGG19 model pre-trained on ImageNet without including the top classification layers
# baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(WIDTH, HEIGHT, CHANNELS)))

In [10]:
# Freeze base model layers
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
# # Enable Transfer Learning by freezing weights of the base VGG16 Model
# for layer in baseModel.layers[:-4]:  # Unfreeze some layers towards the end
#     layer.trainable = True

In [12]:
model = Sequential([
    baseModel,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(CLASSES, activation='softmax')
])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 2587462 (9.87 MB)
Trainable params: 329478 (1.26 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [14]:
from scipy import ndimage


In [15]:

# Data augmentation and normalization for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to [0,1]
    rotation_range=20,           # Randomly rotate images within 20 degrees
    shear_range=0.2,             # Apply shear transformation
    zoom_range=0.2,              # Apply random zoom
    horizontal_flip=True,        # Flip images horizontally
    vertical_flip=True,           # Flip images vertically
    fill_mode='nearest',
)

# Data augmentation and normalization for the validation dataset
valid_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to [0,1]
    rotation_range=20,           # Randomly rotate images within 20 degrees
    shear_range=0.2,             # Apply shear transformation
    zoom_range=0.2,              # Apply random zoom
    horizontal_flip=True,        # Flip images horizontally
    vertical_flip=True,           # Flip images vertically
    fill_mode='nearest',
)

# Normalization for the test dataset (no data augmentation for test data)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0,1] for test data


In [16]:
# Generating batches of preprocessed images and their labels using flow_from_directory
BATCH_SIZE = 32
# Training set generator
training_set = train_datagen.flow_from_directory(
    train_dir,                      # Directory containing training images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical'        # Uses categorical labels for multi-class classification
)

# Validation set generator
validation_set = valid_datagen.flow_from_directory(
    valid_dir,                      # Directory containing validation images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical'        # Uses categorical labels for multi-class classification
)

# Test set generator
test_set = test_datagen.flow_from_directory(
    test_dir,                       # Directory containing test images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical',       # Uses categorical labels for multi-class classification
    shuffle=False                   # Disables shuffling to maintain order in the test set
)


Found 1656 images belonging to 6 classes.
Found 1656 images belonging to 6 classes.
Found 72 images belonging to 6 classes.


In [17]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [18]:
# Training the model using the fit method with training and validation data
Epochs = 5
history = model.fit(
    training_set,                                  # Training dataset
    epochs=Epochs,                                 # Number of training epochs
    steps_per_epoch=training_set.samples // 32,  # Number of steps (batches) per epoch for the training set
    validation_data=validation_set,                # Validation dataset
    validation_steps=validation_set.samples // 32  # Number of steps (batches) per epoch for the validation set
)


Epoch 1/5
51/51 [==============================] - 76s 1s/step - loss: 0.2660 - accuracy: 0.9113 - val_loss: 0.1037 - val_accuracy: 0.9657
Epoch 2/5
51/51 [==============================] - 72s 1s/step - loss: 0.0357 - accuracy: 0.9895 - val_loss: 0.0200 - val_accuracy: 0.9963
Epoch 3/5
51/51 [==============================] - 71s 1s/step - loss: 0.0306 - accuracy: 0.9908 - val_loss: 0.0765 - val_accuracy: 0.9694
Epoch 4/5
51/51 [==============================] - 72s 1s/step - loss: 0.0249 - accuracy: 0.9926 - val_loss: 0.0202 - val_accuracy: 0.9945
Epoch 5/5
51/51 [==============================] - 72s 1s/step - loss: 0.0168 - accuracy: 0.9951 - val_loss: 0.0324 - val_accuracy: 0.9896


In [19]:
# Evaluating the trained model using the test dataset

scores = model.evaluate(test_set)  # Evaluates the model's performance on the test set

3/3 [==============================] - 1s 348ms/step - loss: 0.0102 - accuracy: 1.0000


In [20]:
# Printing the test loss value

print('Test Loss: ', scores[0])  # Prints the computed test loss value


Test Loss:  0.010184993967413902


In [21]:
# Printing the test accuracy

print('Test Accuracy: ', scores[1])  # Prints the computed test accuracy


Test Accuracy:  1.0


In [22]:
model.save('trained_mobilenetv2_model.keras')


In [23]:
# Function to load an image using PIL library

from PIL import Image

def load_image(image_path): # Importing the Image module from the PIL library

    """
    Loads an image using the PIL library.
    
    Args:
    - image_path: The file path of the image to be loaded
    
    Returns:
    - img: The loaded image object
    """

    img = Image.open(image_path)
    return img


In [24]:
from PIL import Image
from keras.preprocessing.image import img_to_array

def preprocess_image(img):
    # Resize the image to the required input size (224x224 for VGG16)
    img = img.resize((224, 224))

    # Convert the image to RGB if it has a single channel (grayscale)
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Convert PIL Image to numpy array
    img_array = img_to_array(img)

    # Normalize pixel values to be in the range [0, 1]
    img_array = img_array / 255.0

    return img_array


In [25]:
from keras.models import load_model
import numpy as np

# Load the saved model
loaded_model = load_model('trained_mobilenetv2_model.keras')


# Use the loaded model to make predictions on new data
def detect_defect(image_path):
    img = load_image(image_path)  # Load the image (implement this function)
    img = preprocess_image(img)   # Preprocess the image (implement this function)

    # Make predictions using the loaded model
    prediction = loaded_model.predict(np.expand_dims(img, axis=0))

    # Check the prediction result
    if prediction[0][0] >= 0.5:  # Assuming binary classification (defect or non-defect)
        return "Defect Detected"
    else:
        return "No Defect Detected"


In [28]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import cv2  # Import OpenCV for image manipulation

# Load the saved model
loaded_model = load_model('trained_mobilenetv2_model.keras')

# Use the loaded model to make predictions on new data
def detect_defect(image_path):
    img = load_image(image_path)  # Load the image (implement this function)
    img = preprocess_image(img)  # Preprocess the image (implement this function)

    # Make predictions using the loaded model
    prediction = loaded_model.predict(np.expand_dims(img, axis=0))

    # Check the prediction result
    if prediction[0][0] >= 0.5:  # Assuming binary classification (defect or non-defect)
        result = "Defect Detected"
    else:
        result = "No Defect Detected"

    # Display the image and the result
    plt.imshow(cv2.imread(image_path))  # Show the original image
    plt.title(result)
    plt.axis('off')  # Hide axis labels
    plt.show()

    return result


In [29]:
# Example usage

image_path = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/test/Crazing/Cr_1.bmp'  # Replace with your image path
detection_result = detect_defect(image_path)
print("Detection Result:", detection_result)

1/1 [==============================] - 1s 664ms/step
Detection Result: Defect Detected


/tmp/ipykernel_11008/1847205834.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [26]:
# # Save the trained model
# model.save('trained_mobilenetv2_model.h5')

In [27]:
# import os
# import tensorflow as tf
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'